# Shakespeare Haiku

In [2]:
import os
import numpy as np
from IPython.display import HTML
import json
from HMM import unsupervised_HMM
from HMM_helper import (
    text_to_wordcloud,
    states_to_wordclouds,
    parse_observations,
    sample_sentence,
    visualize_sparsities,
    animate_emission, 
)
import re
import sys
import random

### Helper functions

In [59]:
def parse_observations_haiku(text):
    """
    Convert text to dataset. 
    """
    lines = [line.split() for line in text.split('\n') if line.split()]

    obs_counter = 0
    obs = []
    obs_map = {}

    for line in lines:
        obs_elem = []
        
        for word in line:
            word = word.strip()
            # Keep apostrophes and hyphens
            word = re.sub(r'[^\w\-\']', '', word).lower() 
            if word not in obs_map:
                # Add unique words to the observations map.
                obs_map[word] = obs_counter
                obs_counter += 1
            
            # Add the encoded word.
            obs_elem.append(obs_map[word])
        
        # Add the encoded sequence.
        obs.append(obs_elem)

    return obs, obs_map

In [90]:
def build_syllables_dict(syllables): 
    """
    Build a syllables dictionary. 
    """
    lines = [line.split() for line in syllables.split('\n') if line.split()]
    syllables_dict = dict()
    for line in lines: 
        word = line[0]
        if len(line) > 2: 
            # Ignore end-of-line syllable count. 
            # Keep the larger syllable count if there are multiple possibilities.
            if "E" in line[1]: 
                syllables_dict[word] = int(line[2])
            elif "E" in line[2]: 
                syllables_dict[word] = int(line[1])
            else: 
                syllables_dict[word] = int(line[2])
        else: 
            syllables_dict[word] = int(line[1])
    return syllables_dict

In [92]:
def obs_map_reverser(obs_map):
    obs_map_r = {}

    for key in obs_map:
        obs_map_r[obs_map[key]] = key

    return obs_map_r

def sample_haiku_line(hmm, obs_map, syllables_dict, n_syllables):
    """
    Generate a sentence with the given number of syllables. 
    """
    obs_map_r = obs_map_reverser(obs_map)
    # Keep generating until we are able to obtain a line 
    # that can be truncated to exactly n_syllables number of 
    # syllables. 
    while True: 
        count = 0
        sentence = []
        emission, states = hmm.generate_emission(10)
        for i in emission: 
            word = obs_map_r[i]
            syllables_count = syllables_dict[word]
            count += syllables_count
            sentence.append(word)
            if count >= n_syllables: 
                break
        if count == n_syllables: 
            break
        
    return ' '.join(sentence).capitalize()

def sample_haiku(hmm, obs_map, syllables_dict): 
    """
    Generate a haiku. 
    """
    first = sample_haiku_line(hmm, obs_map, syllables_dict, 5)
    second = sample_haiku_line(hmm, obs_map, syllables_dict, 7)
    third = sample_haiku_line(hmm, obs_map, syllables_dict, 5)
    return first + '\n' + second + '\n' + third

### HMM Model

In [88]:
syllables = open(os.path.join(os.getcwd(), 'data_Shakespeare/Syllable_dictionary.txt')).read()

In [91]:
syllables = open(os.path.join(os.getcwd(), 'data_Shakespeare/Syllable_dictionary.txt')).read()
syllables_dict = build_syllables_dict(syllables)

In [60]:
text = open(os.path.join(os.getcwd(), 'data_Shakespeare/shakespeare.txt')).read()
obs, obs_map = parse_observations_haiku(text)

In [63]:
hmm = unsupervised_HMM(obs, 16, 100)

Iteration: 10
Iteration: 20
Iteration: 30
Iteration: 40
Iteration: 50
Iteration: 60
Iteration: 70
Iteration: 80
Iteration: 90
Iteration: 100


### Generate Haikus

In [41]:
print(sample_haiku(hmm, obs_map, syllables_dict))

Art or his beauty
Though in will than in me not
Clouds him shall habit


In [96]:
print(sample_haiku(hmm, obs_map, syllables_dict))

I in a-doting
See love who thou sky muse it
Determination


In [97]:
print(sample_haiku(hmm, obs_map, syllables_dict))

How shady her will
Or onset any gift a
Have and still lies bred


In [106]:
print(sample_haiku(hmm, obs_map, syllables_dict))

What witness lends thy
His own you nor can you place
Wilfully our


In [108]:
print(sample_haiku(hmm, obs_map, syllables_dict))

Hath no having time
And of you to these on the
Lived love pen this love
